In [1]:
import os

import tensorflow as tf
from tensorflow import keras

In [2]:
tf.__version__

'1.10.0'

In [3]:
CHECKPOINT_PATH = "checkpoints/save_and_restore_cp.ckpt"
CHECKPOINT_PATH2 = "checkpoints2/save_and_restore_cp-{epoch:04d}.ckpt"
MANUAL_SAVE_PATH = "checkpoints3/manual_save"
H5_PATH = "h5_saves/my_model.h5"
NUM_EPOCHS = 10

In [4]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

In [5]:
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [6]:
# define a simple model
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
        keras.layers.Dropout(0.02),
        keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=["accuracy"], optimizer="adam")
    return model

In [7]:
model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Checkpoints

In [8]:
checkpoint_dir = os.path.dirname(CHECKPOINT_PATH)
checkpoint_dir

'checkpoints'

In [9]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(CHECKPOINT_PATH, save_weights_only=True, verbose=1)

model = create_model()
model.fit(train_images, train_labels, callbacks=[cp_callback], epochs=NUM_EPOCHS, validation_data=(test_images, test_labels))

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 1s 750us/step - loss: 1.1042 - acc: 0.6930 - val_loss: 0.6992 - val_acc: 0.7700

Epoch 00001: saving model to checkpoints/save_and_restore_cp.ckpt
Epoch 2/10
1000/1000 [==============================] - 0s 228us/step - loss: 0.3838 - acc: 0.8950 - val_loss: 0.5151 - val_acc: 0.8400

Epoch 00002: saving model to checkpoints/save_and_restore_cp.ckpt
Epoch 3/10
1000/1000 [==============================] - 0s 230us/step - loss: 0.2592 - acc: 0.9380 - val_loss: 0.4790 - val_acc: 0.8520

Epoch 00003: saving model to checkpoints/save_and_restore_cp.ckpt
Epoch 4/10
1000/1000 [==============================] - 0s 220us/step - loss: 0.1824 - acc: 0.9580 - val_loss: 0.4211 - val_acc: 0.8610

Epoch 00004: saving model to checkpoints/save_and_restore_cp.ckpt
Epoch 5/10
1000/1000 [==============================] - 0s 215us/step - loss: 0.1212 - acc: 0.9820 - val_loss: 0.4122 - val_acc: 0.8650

Epo

In [10]:
!ls {checkpoint_dir}

checkpoint				      save_and_restore_cp.ckpt.index
save_and_restore_cp.ckpt.data-00000-of-00001


Evaluate an untrained model (low accuracy).

In [11]:
model = create_model()
loss, acc = model.evaluate(test_images, test_labels)

print("Untrained model, accuracy: {:5.2f}%".format(100.0 * acc))

1000/1000 [==============================] - 0s 167us/step
Untrained model, accuracy:  9.00%


Load weights from checkpoint and re-evaluate.

In [12]:
model.load_weights(CHECKPOINT_PATH)
loss, acc = model.evaluate(test_images, test_labels)

print("Restored model, accuracy: {:5.2f}%".format(100.0 * acc))

1000/1000 [==============================] - 0s 37us/step
Restored model, accuracy: 86.20%


### Detailed checkpoint usage

In [13]:
checkpoint_dir = os.path.dirname(CHECKPOINT_PATH2)
checkpoint_dir

'checkpoints2'

In [14]:
# configure to save weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(CHECKPOINT_PATH2, period=5, save_weights_only=True, verbose=1)

model = create_model()
model.fit(train_images, train_labels, callbacks=[cp_callback], epochs=NUM_EPOCHS * 2, validation_data=(test_images, test_labels))

Train on 1000 samples, validate on 1000 samples
Epoch 1/20
1000/1000 [==============================] - 1s 730us/step - loss: 1.1024 - acc: 0.7040 - val_loss: 0.6897 - val_acc: 0.7860
Epoch 2/20
1000/1000 [==============================] - 0s 257us/step - loss: 0.3914 - acc: 0.8930 - val_loss: 0.5784 - val_acc: 0.8080
Epoch 3/20
1000/1000 [==============================] - 0s 223us/step - loss: 0.2669 - acc: 0.9240 - val_loss: 0.4986 - val_acc: 0.8370
Epoch 4/20
1000/1000 [==============================] - 0s 222us/step - loss: 0.1737 - acc: 0.9580 - val_loss: 0.4484 - val_acc: 0.8580
Epoch 5/20
1000/1000 [==============================] - 0s 214us/step - loss: 0.1251 - acc: 0.9760 - val_loss: 0.4486 - val_acc: 0.8630

Epoch 00005: saving model to checkpoints2/save_and_restore_cp-0005.ckpt
Epoch 6/20
1000/1000 [==============================] - 0s 230us/step - loss: 0.0869 - acc: 0.9890 - val_loss: 0.4285 - val_acc: 0.8670
Epoch 7/20
1000/1000 [==============================] - 0s 226u

In [15]:
!ls {checkpoint_dir}

checkpoint
save_and_restore_cp-0005.ckpt.data-00000-of-00001
save_and_restore_cp-0005.ckpt.index
save_and_restore_cp-0010.ckpt.data-00000-of-00001
save_and_restore_cp-0010.ckpt.index
save_and_restore_cp-0015.ckpt.data-00000-of-00001
save_and_restore_cp-0015.ckpt.index
save_and_restore_cp-0020.ckpt.data-00000-of-00001
save_and_restore_cp-0020.ckpt.index


Restore latest checkpoint from file.

In [16]:
latest_checkpoint_file = tf.train.latest_checkpoint(checkpoint_dir)
latest_checkpoint_file

'checkpoints2/save_and_restore_cp-0020.ckpt'

In [17]:
model = create_model()
model.load_weights(latest_checkpoint_file)
loss, acc = model.evaluate(test_images, test_labels)

print("Restored model, accuracy: {:5.2f}%".format(100.0 * acc))

1000/1000 [==============================] - 0s 118us/step
Restored model, accuracy: 87.00%


## Manual Saving

In [18]:
model.save_weights(MANUAL_SAVE_PATH)

In [19]:
!ls checkpoints3

checkpoint  manual_save.data-00000-of-00001  manual_save.index


In [20]:
# new model
model = create_model()
# restore
model.load_weights(MANUAL_SAVE_PATH)

loss, acc = model.evaluate(test_images, test_labels)

print("Manually restored model, accuracy: {:5.2f}%".format(100.0 * acc))

1000/1000 [==============================] - 0s 125us/step
Manually restored model, accuracy: 87.00%


## Saving Entire Models

In [21]:
!mkdir h5_saves

mkdir: cannot create directory ‘h5_saves’: File exists


Save to H5 format.

In [22]:
model = create_model()
model.fit(train_images, train_labels, epochs=int(NUM_EPOCHS/2.0))
model.save(H5_PATH)

Epoch 1/5
1000/1000 [==============================] - 1s 671us/step - loss: 1.0878 - acc: 0.6940
Epoch 2/5
1000/1000 [==============================] - 0s 188us/step - loss: 0.3924 - acc: 0.8910
Epoch 3/5
1000/1000 [==============================] - 0s 228us/step - loss: 0.2457 - acc: 0.9370
Epoch 4/5
1000/1000 [==============================] - 0s 317us/step - loss: 0.1844 - acc: 0.9600
Epoch 5/5
1000/1000 [==============================] - 2s 2ms/step - loss: 0.1242 - acc: 0.9790


In [23]:
!ls h5_saves

my_model.h5


Load saved model.

In [24]:
new_model = keras.models.load_model(H5_PATH)
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [25]:
loss, acc = new_model.evaluate(test_images, test_labels)

print("Fully restored model, accuracy: {:5.2f}%".format(100.0 * acc))

1000/1000 [==============================] - 0s 150us/step
Fully restored model, accuracy: 85.80%
